In [ ]:
"""
All the imports needed throughout the project, the data is mounted onto the notebook via google drive mounting.
"""

import time
import copy
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
multiprocessing.set_start_method('spawn', force=True)

import random
import numpy as np
import pandas as pd
from google.colab import drive

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.amp import autocast, GradScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from tqdm.auto import tqdm

drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)
# !ls "/content/drive/My Drive/Foundations AI Final Project Data/"

torch.backends.cudnn.benchmark = True


In [2]:
"""
NHANES Data Processing
"""

xpt_2023_demographic = '/content/drive/My Drive/Foundations AI Final Project Data/2023_demographic.xpt'
xpt_2020_demographic = '/content/drive/My Drive/Foundations AI Final Project Data/2020_demographic.xpt'
xpt_2016_demographic = '/content/drive/My Drive/Foundations AI Final Project Data/2016_demographic.xpt'
items1 = ["RIAGENDR", "RIDAGEYR", "RIDRETH1"]


xpt_2023_diabetes = '/content/drive/My Drive/Foundations AI Final Project Data/2023_diabetes.xpt'
xpt_2020_diabetes = '/content/drive/My Drive/Foundations AI Final Project Data/2020_diabetes.xpt'
xpt_2016_diabetes = '/content/drive/My Drive/Foundations AI Final Project Data/2016_diabetes.xpt'
items2 = ["DIQ010"]

xpt_2023_body = '/content/drive/My Drive/Foundations AI Final Project Data/2023_body.xpt'
xpt_2020_body = '/content/drive/My Drive/Foundations AI Final Project Data/2020_body.xpt'
xpt_2016_body = '/content/drive/My Drive/Foundations AI Final Project Data/2016_body.xpt'
items3 = ["BMXWT", "BMXHT", "BMXBMI", "BMXWAIST", "BMXARMC"]

xpt_2023_glyco = '/content/drive/My Drive/Foundations AI Final Project Data/2023_glyco.xpt'
xpt_2020_glyco = '/content/drive/My Drive/Foundations AI Final Project Data/2020_glyco.xpt'
xpt_2016_glyco = '/content/drive/My Drive/Foundations AI Final Project Data/2016_glyco.xpt'
items4 = ["LBXGH"]

xpt_2023_glucose = '/content/drive/My Drive/Foundations AI Final Project Data/2023_glucose.xpt'
xpt_2020_glucose = '/content/drive/My Drive/Foundations AI Final Project Data/2020_glucose.xpt'
xpt_2016_glucose = '/content/drive/My Drive/Foundations AI Final Project Data/2016_glucose.xpt'
items5 = ["LBXGLU"]

xpt_2023_insulin = '/content/drive/My Drive/Foundations AI Final Project Data/2023_insulin.xpt'
xpt_2020_insulin = '/content/drive/My Drive/Foundations AI Final Project Data/2020_insulin.xpt'
xpt_2016_insulin = '/content/drive/My Drive/Foundations AI Final Project Data/2016_insulin.xpt'
items6 = ["LBXIN"]

xpt_2023_dietary = '/content/drive/My Drive/Foundations AI Final Project Data/2023_dietary.xpt'
xpt_2020_dietary = '/content/drive/My Drive/Foundations AI Final Project Data/2020_dietary.xpt'
xpt_2016_dietary = '/content/drive/My Drive/Foundations AI Final Project Data/2016_dietary.xpt'
items7 = ["DRQSPREP", "DR1TSUGR", "DR1TCARB", "DR1TTFAT"]

xpt_2023_bp_c = '/content/drive/My Drive/Foundations AI Final Project Data/2023_bp_c.xpt'
xpt_2020_bp_c = '/content/drive/My Drive/Foundations AI Final Project Data/2020_bp_c.xpt'
xpt_2016_bp_c = '/content/drive/My Drive/Foundations AI Final Project Data/2016_bp_c.xpt'
items8 = ["BPQ020", "BPQ080"]

xpt_2023_hospital = '/content/drive/My Drive/Foundations AI Final Project Data/2023_hospital.xpt'
xpt_2020_hospital = '/content/drive/My Drive/Foundations AI Final Project Data/2020_hospital.xpt'
xpt_2016_hospital = '/content/drive/My Drive/Foundations AI Final Project Data/2016_hospital.xpt'
items9 = ["HUQ010"]

xpt_2023_smoke = '/content/drive/My Drive/Foundations AI Final Project Data/2023_smoke.xpt'
xpt_2020_smoke = '/content/drive/My Drive/Foundations AI Final Project Data/2020_smoke.xpt'
xpt_2016_smoke = '/content/drive/My Drive/Foundations AI Final Project Data/2016_smoke.xpt'
items10 = ["SMD460"]

# SEQN - Respondent sequence number
# DIQ010 - Have diabetes [1 yes, 2 no]
# RIAGENDR - Gender
# RIDAGEYR - Age
# RIDRETH1 - Ethinicity [C]
# BMXWT - Weight kg
# BMXHT - Height cm
# BMXBMI - BMI
# BMXWAIST - Waist circumference cm
# BMXARMC - Arm circumference cm
# LBXGH - Glycohemoglobin (%)
# LBXGLU - Fasting Glucose (mg/dL)
# LBXIN - Insulin (uU/mL)
# DRQSPREP - Salt used in preparation [1 never ~ 4 very often, 9 donotknow, . missing]
# DR1TSUGR - Total sugars (gm)
# DR1TCARB - Carbohydrate (gm)
# DR1TTFAT - Total fat (gm)
# BPQ020 - Have high blood pressure [1 yes, 2 no]
# BPQ080 = Have high cholesterol [1 yes, 2 no]
# HUQ010 - General health condition [1 ~ 5, 1 is best]
# SMD460 - Number of people who live here smoke tobacco [0, 1, 2]



files = {
    "demographic": ["RIAGENDR", "RIDAGEYR", "RIDRETH1"],
    "body": ["BMXWT", "BMXHT", "BMXBMI", "BMXWAIST", "BMXARMC"],
    "glyco": ["LBXGH"],
    "glucose": ["LBXGLU"],
    "insulin": ["LBXIN"],
    "dietary": ["DRQSPREP", "DR1TSUGR", "DR1TCARB", "DR1TTFAT"],
    "bp_c": ["BPQ020", "BPQ080"],
    "hospital": ["HUQ010"],
    "smoke": ["SMD460"]
}

df_2016 = pd.read_sas(xpt_2016_diabetes, format='xport')[['SEQN','DIQ010']]
df_2020 = pd.read_sas(xpt_2020_diabetes, format='xport')[['SEQN','DIQ010']]
df_2023 = pd.read_sas(xpt_2023_diabetes, format='xport')[['SEQN','DIQ010']]

# There shows no duplicates in the SEQN numbers
df_persons = pd.concat([df_2016, df_2020, df_2023], ignore_index=True)
df_persons = df_persons[df_persons["DIQ010"].isin([1, 2])]

'\nNHANES Data Processing\n'

In [2]:
# Extract the rows and columns from the other files

file_base_path = "/content/drive/My Drive/Foundations AI Final Project Data/"
df_main = df_persons.set_index("SEQN")

for year in ["2016","2020","2023"]:
    for name, cols in files.items():
        path = f"{file_base_path}{year}_{name}.xpt"
        df_year = (
            pd.read_sas(path, format="xport")
              .set_index("SEQN")[cols]
              .reindex(df_main.index)
        )
        for c in cols:
            if c in df_main.columns:
                df_main[c] = df_main[c].fillna(df_year[c])
            else:
                df_main[c] = df_year[c]

df_main = df_main.reset_index()


# Target encoding
# This must be kept in the data frame init cell as repeated runs change the data

df_main['DIQ010'] = (df_main['DIQ010'] == 1).astype(int)
df_main['RIAGENDR'] = (df_main['RIAGENDR'] == 1).astype(int)
df_main['RIDRETH1'] = df_main.groupby('RIDRETH1')['DIQ010'].transform('mean')
df_main['BPQ020'] = (df_main['BPQ020'] == 1).astype(int)
df_main['BPQ080'] = (df_main['BPQ080'] == 1).astype(int)

col = df_main['DRQSPREP'].where(df_main['DRQSPREP'].between(1,4))
df_main['DRQSPREP'] = (col.fillna(col.median()) - 1) / 3

col = df_main['HUQ010'].where(df_main['HUQ010'].between(1,5))
df_main['HUQ010'] = (col.fillna(5 - col.median())) / 4

cols = ['RIDAGEYR', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXWAIST', 'BMXARMC', 'LBXGH', 'LBXGLU', 'LBXIN', 'DR1TSUGR', 'DR1TCARB', 'DR1TTFAT']
df_main[cols] = df_main[cols].apply(lambda x: (x.fillna(x.median()) - x.min()) / (x.max() - x.min()))

rounded = df_main['SMD460'].round()
med = rounded.median()
df_main['SMD460'] = rounded.fillna(med).astype('Int64')


df_main.drop('SEQN', axis=1, inplace=True)
df_main.rename(columns={'DIQ010': 'Diabetes_binary'}, inplace=True)

n_pos = df_main['Diabetes_binary'].eq(1).sum()
df_pos = df_main[df_main['Diabetes_binary'] == 1]
df_neg_sample = df_main[df_main['Diabetes_binary'] == 0].sample(n=n_pos, random_state=42)
df_balanced = pd.concat([df_pos, df_neg_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
"""
Kaggle data processing
"""

kaggle_data_path = "/content/drive/My Drive/Foundations AI Final Project Data/kaggle_diabetes_data.csv"
kaggle_data_df = pd.read_csv(kaggle_data_path)

for col in kaggle_data_df.columns:
    max_val = kaggle_data_df[col].max()
    if max_val > 1:
        kaggle_data_df[col] = kaggle_data_df[col] / max_val

In [ ]:
"""
The base GA-NN training algorithm
"""

class GeneticAlgorithm:
    def __init__(
            self,
            data = kaggle_data_df,
            n_features = 21,
            pop_size = 24,
            generations = 120,
            fitness_threshold = 0.9,
            crossover_rate = 0.8,
            mutation_rate = 0.1,
            epoch_limit = 8,
            batch_size = 3200,
            learning_rate = 0.01
            ):

        self.data = data
        self.n_features = n_features
        self.pop_size = pop_size
        self.generations = generations
        self.fitness_threshold = fitness_threshold
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate

        self.epoch_limit = epoch_limit
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        self.population = self.initialize_population()
        self.pop_fitness = []

    def initialize_population(self):
        pop = np.random.randint(0, 2, (self.pop_size-1, self.n_features))
        full_ind = np.ones((1, self.n_features), dtype=int)
        return np.concatenate((pop, full_ind), axis=0)

    def selection(self, fitnesses):
        chosen = []
        for _ in range(self.pop_size):
            i, j = random.sample(range(self.pop_size), 2)
            if fitnesses[i] > fitnesses[j]:
                chosen.append(self.population[i].copy())
            else:
                chosen.append(self.population[j].copy())
        return np.array(chosen)

    def crossover(self, p1, p2):
        i = random.randint(1, self.n_features - 1)
        c1 = np.concatenate((p1[:i], p2[i:]))
        c2 = np.concatenate((p2[:i], p1[i:]))
        return c1, c2

    def mutate(self, indv):
        for i in range(self.n_features):
            if random.random() < self.mutation_rate:
                # flips the boolean indicator value
                indv[i] = 1 - indv[i]
        return indv

    def next_gen(self):
        self.evaluate_population()
        chosen = self.selection(self.pop_fitness)
        new_pop = []

        for i in range(0, self.pop_size, 2):
            p1 = chosen[i]
            p2 = chosen[ i+1 if i+1 < self.pop_size else 0 ]

            if random.random() < self.crossover_rate:
                c1, c2 = self.crossover(p1, p2)
            else:
                c1, c2 = p1.copy(), p2.copy()

            new_pop.append(self.mutate(c1))
            new_pop.append(self.mutate(c2))
        # prevents population size increase due to even number of children despite odd population size
        self.population = np.array(new_pop[:self.pop_size])

    def evolve(self):
        for gen in tqdm(range(self.generations), desc='GA Generations'):
            self.next_gen()
            print("Generation", gen, "Best fit:", max(self.pop_fitness))
        self.evaluate_population()
        print("Final fitness", self.pop_fitness)
        print("Best indv", self.population[np.argmax(self.pop_fitness)])


    def evaluate_gene(self, gene):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        selected_features = np.where(gene == 1)[0]
        if len(selected_features) < 5:
            return 0.0

        data_sample = self.data.sample(n=30000)
        X = data_sample.drop("Diabetes_binary", axis=1).values
        y = data_sample["Diabetes_binary"].values
        X_selected = X[:, selected_features]
        X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)

        model = nn.Sequential(
            nn.Linear(X_train_tensor.shape[1], 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        ).to(device)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)

        for epoch in tqdm(range(self.epoch_limit), desc='Training NN', leave=False, position=0):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                preds = model(X_batch)
                loss = criterion(preds, y_batch)
                loss.backward()
                optimizer.step()
        model.eval()

        with torch.no_grad():
            outputs = model(X_test_tensor)
            preds = (outputs > 0.5).float()
            accuracy = (preds == y_test_tensor).sum().item() / y_test_tensor.shape[0]

            # Compute AUC score:
            pred_probs = outputs.detach().cpu().numpy()
            y_true = y_test_tensor.detach().cpu().numpy()
            try:
                auc = roc_auc_score(y_true, pred_probs)
            except Exception as e:
                # Fallback if AUC calculation fails (e.g., if one class is missing)
                auc = accuracy

        """
        This version utilized mainly auc and acc metrics and various ways to combine them
        """

        weight_acc = 1.1
        weight_auc = 0.9
        composite = (accuracy ** weight_acc) * (auc ** weight_auc)

        penalty_weight = 0.1
        penalty = penalty_weight * ((len(selected_features) / self.n_features) ** 2)

        final_fitness = composite - penalty
        return final_fitness


    def evaluate_population(self):
        # Use ProcessPoolExecutor.map, which returns results in order of the input.
        with ThreadPoolExecutor(max_workers=12) as executor:
        # The executor.map call will preserve order.
            self.pop_fitness = list(tqdm(executor.map(self.evaluate_gene, self.population),
                                        total=len(self.population),
                                        desc="Evaluating NNs",
                                        leave=False,
                                        position=0))

In [ ]:
class GeneticAlgorithm_v2:
    def __init__(
            self,
            data,
            n_features = 21,
            pop_size = 32,
            generations = 200,
            fitness_threshold = 0.9,
            crossover_rate = 0.5,
            mutation_rate = 0.1,
            epoch_limit = 8,
            batch_size = 3200,
            learning_rate = 0.01,
            children_count = 8,
            ):

        """
        To run the cooperative coevalutation version, reduce the n_features to the gene size, and add adjustments before the NN training to align the features
        """

        self.data = data
        self.n_features = n_features
        self.pop_size = pop_size
        self.generations = generations
        self.fitness_threshold = fitness_threshold
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate

        self.epoch_limit = epoch_limit
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        self.population = self.initialize_population()
        self.pop_fitness = []

        self.children_count = children_count
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def initialize_population(self):
        pop = np.random.randint(0, 2, (self.pop_size-1, self.n_features))
        full_ind = np.ones((1, self.n_features), dtype=int)
        return np.concatenate((pop, full_ind), axis=0)

    def selection(self, fitnesses):
        scaled = np.array(fitnesses) - np.min(fitnesses) + 1e-6
        probs  = scaled / scaled.sum()
        idxs   = np.random.choice(self.pop_size, size=2, p=probs)
        return self.population[idxs].copy()

    def crossover(self, p1, p2):
        c1, c2 = p1.copy(), p2.copy()
        for i in range(self.n_features):
            if random.random() > self.crossover_rate:
                c1[i], c2[i] = c2[i], c1[i]
        return c1, c2

    def mutate(self, indv):
        for i in range(self.n_features):
            if random.random() < self.mutation_rate:
                # flips the boolean indicator value
                indv[i] = 1 - indv[i]
        return indv

    def next_gen(self):
        children = []
        for _ in range(self.children_count // 2):
            p1, p2 = self.selection(self.pop_fitness)
            c1, c2 = self.crossover(p1, p2)
            children.append(self.mutate(c1))
            children.append(self.mutate(c2))

        combined_population = np.concatenate((self.population, children), axis=0)
        new_fitness = self.evaluate_group(children)
        combined_fitness = np.concatenate((self.pop_fitness, new_fitness), axis=0)

        # Select the best individuals from the combined population
        best_indices = np.argsort(combined_fitness)[-self.pop_size:]

        # Replace the population with the best individuals
        self.population = combined_population[best_indices]
        self.pop_fitness = combined_fitness[best_indices]

    def evolve(self):
        self.evaluate_population()
        for gen in tqdm(range(self.generations), desc='GA Generations'):
            self.next_gen()
            # Find index of best fitness in current population
            best_idx = int(np.argmax(self.pop_fitness))
            best_fit = self.pop_fitness[best_idx]
            best_ind = self.population[best_idx]
            # Print both fitness and the binary vector of the best individual
            print(f"Generation {gen} | Best fit: {best_fit:.4f} | Best indv: {best_ind}")
        # Final summary
        final_best_idx = int(np.argmax(self.pop_fitness))
        print("Final fitness:", self.pop_fitness)
        print("Best individual overall:", self.population[final_best_idx])


    def evaluate_gene(self, gene):
        selected_features = np.where(gene == 1)[0]
        if len(selected_features) < 4:
            return 0.0

        X = self.data.drop("Diabetes_binary", axis=1).to_numpy(dtype=np.float32)
        y = self.data["Diabetes_binary"].to_numpy(dtype=np.float32)
        X_selected = X[:, selected_features]
        X_train_all, X_test, y_train_all, y_test = train_test_split(
            X_selected, y, test_size=0.3, random_state=42
        )

        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(self.device, non_blocking=True)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(self.device, non_blocking=True)


        kf = KFold(n_splits=4, shuffle=True, random_state=42)
        fold_scores = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_all), 1):
            X_train, y_train = X_train_all[train_idx], y_train_all[train_idx]
            X_val,   y_val   = X_train_all[val_idx],   y_train_all[val_idx]

            idx = np.random.choice(len(X_train), size=4096, replace=False)
            X_train, y_train = X_train[idx], y_train[idx]

            # CPU tensors → DataLoader
            X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
            y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
            X_val_tensor   = torch.tensor(X_val,   dtype=torch.float32).to(self.device, non_blocking=True)
            y_val_tensor   = torch.tensor(y_val,   dtype=torch.float32).unsqueeze(1).to(self.device, non_blocking=True)

            train_loader = DataLoader(
                TensorDataset(X_train_tensor, y_train_tensor),
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=4,
                pin_memory=True
            )

            model = nn.Sequential(
                nn.Linear(X_train_tensor.shape[1], 16),
                nn.ReLU(),
                nn.Dropout(0.25),
                nn.Linear(16, 1)
            ).to(self.device)

            criterion = nn.BCEWithLogitsLoss()
            optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

            scaler = GradScaler()

            for epoch in tqdm(range(self.epoch_limit), desc='Training NN', leave=False, position=0):
                t0 = time.time()

                # — TRAINING —
                model.train()
                for Xb, yb in train_loader:
                    Xb, yb = Xb.to(self.device, non_blocking=True), yb.to(self.device, non_blocking=True)
                    optimizer.zero_grad()
                    with autocast(device_type='cuda'):
                        logits = model(Xb)
                        loss   = criterion(logits, yb)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                # — VALIDATE (use X_val) —
                model.eval()
                with torch.no_grad():
                    val_out  = model(X_val_tensor)
                    val_loss = criterion(val_out, y_val_tensor)
                scheduler.step(val_loss)

            model.eval()
            with torch.no_grad():
                test_logits = model(X_test_tensor)
                test_probs  = torch.sigmoid(test_logits)

                mse = ((test_probs - y_test_tensor)**2).mean().item()
                preds    = (test_probs > 0.5).float()
                acc      = (preds == y_test_tensor).float().mean().item()
                auc      = roc_auc_score(
                            y_test_tensor.cpu().numpy(),
                            test_probs.cpu().numpy()
                        )

            penalty = 0
            if np.sum(gene) > 10:
                penalty = ((np.sum(gene) - 10) / (self.n_features - 10)) * 0.05
            fold_scores.append(0.4 * acc + 0.4 * auc + 0.2 * (1 - mse) - penalty)

        return float(np.mean(fold_scores))

    """
    Parallel COmputing setup for faster training
    """

    def evaluate_group(self, group):
        with ThreadPoolExecutor(max_workers=8) as executor:
            # Execute evaluate_gene in parallel for the group (which can be the children)
            return list(tqdm(executor.map(self.evaluate_gene, group),
                            total=len(group),
                            desc="Evaluating new NNs",
                            leave=False,
                            position=0))

    def evaluate_population(self):
        # Use ProcessPoolExecutor.map, which returns results in order of the input.
        with ThreadPoolExecutor(max_workers=12) as executor:
        # The executor.map call will preserve order.
            self.pop_fitness = list(tqdm(executor.map(self.evaluate_gene, self.population),
                                        total=len(self.population),
                                        desc="Evaluating NNs",
                                        leave=False,
                                        position=0))


In [ ]:
"""
Specific code that trains a specific feature combination
"""

def parse_gene_str(gene_str):
    return [i for i, b in enumerate(gene_str.split()) if b == '1']

def train_nn_model(df, selected_features, test_size=0.3, epoch_limit=100, batch_size=1600, learning_rate=0.005):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X = df[selected_features].to_numpy(dtype=np.float32)
    y = df["Diabetes_binary"].to_numpy(dtype=np.float32)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train_t = torch.tensor(X_train).to(device)
    y_train_t = torch.tensor(y_train).unsqueeze(1).to(device)
    X_test_t  = torch.tensor(X_test).to(device)
    y_test_t  = torch.tensor(y_test).unsqueeze(1).to(device)
    train_dl = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=True)
    model = nn.Sequential(
        nn.Linear(len(selected_features), 16),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(16, 1)
    ).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for _ in range(epoch_limit):
        model.train()
        for xb, yb in train_dl:
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()
        print("new epoch")
    model.eval()
    with torch.no_grad():
        probs = torch.sigmoid(model(X_test_t))
        preds = (probs > 0.5).float()
        acc = (preds == y_test_t).float().mean().item()
        auc = roc_auc_score(y_test, probs.cpu().numpy())
    return model, {"accuracy": acc, "auc": auc}

def train_from_gene_str(df, gene_str, **kwargs):
    feature_cols = [c for c in df.columns if c != "Diabetes_binary"]
    idxs = parse_gene_str(gene_str)
    selected = [feature_cols[i] for i in idxs]
    return train_nn_model(df, selected, **kwargs)